# Big Data Modeling and Management 2021


## 🚚 BDMM Third Homework Assignment 🚚 

### Part 2 - Queries

In [1]:
from pymongo import MongoClient, GEOSPHERE
from bson.objectid import ObjectId
from pprint import pprint
import numpy as np


host="rhea.isegi.unl.pt"
port="27039"
user="GROUP_22"
password="AW319u0374I82895091g8k9291991eJV"
protocol="mongodb"
database = 'WWI'
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [2]:
db = client.WWI
print(f"Database info: {db}\n")

Database info: Database(MongoClient(host=['rhea.isegi.unl.pt:27039'], document_class=dict, tz_aware=False, connect=True), 'WWI')



In [3]:
# check to see if they all embeddings were done correctly
collection_list = db.list_collection_names()
print(f"The database contains {len(collection_list)} collections")

print(f"All collections: {collection_list}")

The database contains 14collections
All collections: ['Project_WWI_Application_People', 'Purchasing_Suppliers', 'Sales_Orders', 'Purchasing_PurchaseOrderLines', 'Sales_Customers', 'Purchasing_PurchaseOrders', 'Sales_Invoices', 'Purchasing_SupplierTransactions', 'Warehouse_StockItemTransactions', 'Transactions', 'Sales_InvoiceLines', 'Warehouse_StockItems', 'Sales_CustomerTransactions', 'Sales_OrderLines']


## The web team needs:

#### From which state province are our suppliers from?

In [4]:
db.Purchasing_Suppliers.distinct('DeliveryCity_Details.State_Province_Details.StateProvinceName')

['California',
 'Indiana',
 'Kentucky',
 'Minnesota',
 'Missouri',
 'New Jersey',
 'North Carolina',
 'South Dakota',
 'Tennessee',
 'Washington']

#### From which state province are the customers who have a higher credit limit?

In [5]:
notna = {'$match': {'CreditLimit': {'$ne' : np.nan}}}

convert = { "$addFields": {
        "CreditLimit": { 
            "$convert": { 
                "input": "$CreditLimit", 
                "to": "int" } } } }

group={ '$group': {
        '_id': {'StateProvinceName':'$DeliveryCity_Details.State_Province_Details.StateProvinceName'},
        'avg': { '$avg': '$CreditLimit' }}}

sort =  {'$sort': {'avg':-1}}

pipeline = [notna,convert,group,sort]

r = db.Sales_Customers.aggregate(pipeline)
result = list(r)
result

[{'_id': {'StateProvinceName': 'Nevada'}, 'avg': 3600.0},
 {'_id': {'StateProvinceName': 'Oregon'}, 'avg': 3530.0},
 {'_id': {'StateProvinceName': 'Connecticut'}, 'avg': 3500.0},
 {'_id': {'StateProvinceName': 'Tennessee'}, 'avg': 3465.0},
 {'_id': {'StateProvinceName': 'Wyoming'}, 'avg': 3365.0},
 {'_id': {'StateProvinceName': 'Utah'}, 'avg': 3200.0},
 {'_id': {'StateProvinceName': 'Indiana'}, 'avg': 3098.75},
 {'_id': {'StateProvinceName': 'Arkansas'}, 'avg': 3044.1666666666665},
 {'_id': {'StateProvinceName': 'Arizona'}, 'avg': 2950.0},
 {'_id': {'StateProvinceName': 'Ohio'}, 'avg': 2942.8571428571427},
 {'_id': {'StateProvinceName': 'Virginia'}, 'avg': 2915.0},
 {'_id': {'StateProvinceName': 'Kansas'}, 'avg': 2900.0},
 {'_id': {'StateProvinceName': 'Alaska'}, 'avg': 2858.3333333333335},
 {'_id': {'StateProvinceName': 'Oklahoma'}, 'avg': 2806.0},
 {'_id': {'StateProvinceName': 'New Hampshire'}, 'avg': 2800.0},
 {'_id': {'StateProvinceName': 'Florida'}, 'avg': 2800.0},
 {'_id': {'Sta

## The warehouse group needs:

#### To know which items get ordered together the most?

In [ ]:
query_1 = {
   '$lookup':
     {
       'from': 'Sales_OrderLines',
       'localField': 'OrderID',
       'foreignField': 'OrderID',
       'as': 'description'
     }
}

unwind = {'$unwind': '$description'}

query_2={
    '$group':{
        '_id':{'item1':'$description.StockItemName', 'item2':'$description.StockItemName'},
        'count':{'$sum':1}
    },
}

query_3={
    '$sort':{
        'count':-1
    }
}

query_4={
    '$limit':5
}

pipeline=[query_1,unwind,query_2,query_3,query_4]

r=db.Sales_OrderLines.aggregate(pipeline)

result=list(r)
result

#### Which items get ordered the most in bulk (bigger amounts)?

In [6]:
query_1 = {
   '$lookup':
     {
       'from': 'Warehouse_StockItems',
       'localField': 'StockItemID',
       'foreignField': 'StockItemID',
       'as': 'description'
     }
}

unwind = {'$unwind': '$description'}

query_2={
    '$group':{
        '_id':{'id':'$description.StockItemID','name':'$description.StockItemName'},
        'count':{'$sum':'$Quantity'}
    },
}


query_3={
    '$sort':{
        'count':-1
    }
}

query_4={
    '$limit':5
}

pipeline=[query_1,unwind,query_2,query_3,query_4]

r=db.Sales_OrderLines.aggregate(pipeline)

result=list(r)
result

[{'_id': {'id': 79, 'name': '"The Gu" red shirt XML tag t-shirt (White) S'},
  'count': 26952},
 {'_id': {'id': 83, 'name': '"The Gu" red shirt XML tag t-shirt (White) XXL'},
  'count': 26688},
 {'_id': {'id': 148, 'name': 'Halloween skull mask (Gray) L'}, 'count': 25104},
 {'_id': {'id': 76, 'name': '"The Gu" red shirt XML tag t-shirt (White) 3XS'},
  'count': 24756},
 {'_id': {'id': 142, 'name': 'Halloween zombie mask (Light Brown) S'},
  'count': 24744}]

#### Which customers have delivery addresses under 10km of distance to the customer with customerID 961?

In [7]:
r = db.Sales_Customers.find_one({'CustomerID':961})
long961 = float(r['DeliveryLocationLong'])
lat961 = float(r['DeliveryLocationLat'])

In [8]:
for document in db.Sales_Customers.find():
    db.Sales_Customers.update(
        {"_id":document["_id"]},
        {"$set": {"PointDeliveryLocation":
                  { "type": "Point", 
                   "coordinates": [ float(document["DeliveryLocationLong"]), float(document["DeliveryLocationLat"])] 
                  }}})

<ipython-input-8-a7f704b3639d>:2: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  db.Sales_Customers.update(


In [9]:
list(db.Sales_Customers.find())[0]

{'_id': ObjectId('60b3d7e5781dea0108ea33f1'),
 'AccountOpenedDate': '2013-01-01',
 'AlternateContactPersonID': 1002.0,
 'BillToCustomerID': 1,
 'CreditLimit': nan,
 'CustomerID': 1,
 'CustomerName': 'Tailspin Toys (Head Office)',
 'Customer_Category': {'CustomerCategoryID': 3,
  'CustomerCategoryName': 'Novelty Shop'},
 'Delivery': {'AddressLine1': 'Shop 38',
  'AddressLine2': '1877 Mittal Road',
  'PostalCode': 90410,
  'Run': nan},
 'DeliveryCity_Details': {'CityID': 19586,
  'CityName': 'Lisco',
  'LatestRecordedPopulation': None,
  'Location': '0xE6100000010CE73F5A52A4BF444010638852B1A759C0',
  'State_Province_Details': {'Country_Details': {'CountryID': 230,
    'CountryName': 'United States',
    'FormalName': 'United States of America',
    'IsoAlpha3Code': 'USA',
    'IsoNumericCode': 840,
    'CountryType': 'UN Member State',
    'LatestRecordedPopulation': 313973000,
    'Continent': 'North America',
    'Region': 'Americas',
    'Subregion': 'Northern America'},
   'LatestRec

In [10]:
db.Sales_Customers.create_index([("PointDeliveryLocation", GEOSPHERE)])

'PointDeliveryLocation_2dsphere'

In [11]:
query_1 = {
    "$geoNear": {
        "near": { "type": "Point", "coordinates": [long961,lat961] },
        "distanceField": "Distancefrom961",
        "spherical": "true",
        "maxDistance": 10000,
        "key": "PointDeliveryLocation"
    }
}
 
query_2 = {
    "$project": {
        "CustomerID":1,
        "CustomerName":1,
        "Distancefrom961":1
    }
}

query_3 = {
    "$sort": {
        "Distancefrom961": 1
    }
}

query_4 = {
    "$skip": 1
}

pipeline = [query_1,query_2,query_3,query_4]
 
r = db.Sales_Customers.aggregate(pipeline)
result = list(r)
result

[{'_id': ObjectId('60b3d7e5781dea0108ea35b6'),
  'CustomerID': 835,
  'CustomerName': 'Dominic Davignon',
  'Distancefrom961': 2846.882560911145},
 {'_id': ObjectId('60b3d7e5781dea0108ea3462'),
  'CustomerID': 846,
  'CustomerName': 'Malorie Bousquet',
  'Distancefrom961': 4178.770457915538}]

## The CFO:

#### Would like to know the monthly order count?

In [12]:
convert = { "$addFields": {
        "OrderDate": { 
            "$convert": { 
                "input": "$OrderDate", 
                "to": "date" } } } }

project = {
      '$addFields':
        { 'year_orderdate': { '$year': "$OrderDate" },
          'month_orderdate': { '$month': "$OrderDate" }}}

group={ '$group': {
        '_id': {'year':'$year_orderdate','month':'$month_orderdate'},
        'count': { '$sum': 1 }}}

sort =  {'$sort': {'_id.year':1,'_id.month':1}}

pipeline = [convert,project,group,sort]

r = db.Sales_Orders.aggregate(pipeline)
result = list(r)
result

[{'_id': {'year': 2013, 'month': 1}, 'count': 1674},
 {'_id': {'year': 2013, 'month': 2}, 'count': 1139},
 {'_id': {'year': 2013, 'month': 3}, 'count': 1683},
 {'_id': {'year': 2013, 'month': 4}, 'count': 1696},
 {'_id': {'year': 2013, 'month': 5}, 'count': 1808},
 {'_id': {'year': 2013, 'month': 6}, 'count': 1675},
 {'_id': {'year': 2013, 'month': 7}, 'count': 1886},
 {'_id': {'year': 2013, 'month': 8}, 'count': 1537},
 {'_id': {'year': 2013, 'month': 9}, 'count': 1617},
 {'_id': {'year': 2013, 'month': 10}, 'count': 1618},
 {'_id': {'year': 2013, 'month': 11}, 'count': 1552},
 {'_id': {'year': 2013, 'month': 12}, 'count': 1565},
 {'_id': {'year': 2014, 'month': 1}, 'count': 1791},
 {'_id': {'year': 2014, 'month': 2}, 'count': 1538},
 {'_id': {'year': 2014, 'month': 3}, 'count': 1586},
 {'_id': {'year': 2014, 'month': 4}, 'count': 1739},
 {'_id': {'year': 2014, 'month': 5}, 'count': 1908},
 {'_id': {'year': 2014, 'month': 6}, 'count': 1887},
 {'_id': {'year': 2014, 'month': 7}, 'count

#### Would like to know the average monthly sales prices?


In [ ]:
convert = { "$addFields": {
        "OrderDate": { 
            "$convert": { 
                "input": "$OrderDate", 
                "to": "date"}}}}

project = {'$addFields':
        { 'year_orderdate': { '$year': "$OrderDate" },
          'month_orderdate': { '$month': "$OrderDate" }}}

lookup = {'$lookup':{
       'from': 'Sales_OrderLines',
       'localField': 'OrderLineID',
       'foreignField': 'OrderID',
       'as': 'Lines'}}

unwind = {'$unwind': '$Lines'}

group={'$group':{
        '_id':{'name':'$Lines.TotalItemPrice','year':'$year_orderdate','month':'$month_orderdate'},
        'avg':{'$avg':'$Lines.TotalItemPrice'}}}

sort={'$sort':{
      'avg':-1}}

pipeline=[convert,project,lookup,unwind,group,sort]
r=db.Sales_Orders.aggregate(pipeline)
result=list(r)

#### Would like to know the yearly expenditures with suppliers (per supplier name)?

In [13]:
convert = { "$addFields": {
        "TransactionDate": { 
            "$convert": { 
                "input": "$TransactionDate", 
                "to": "date"}}}}

addfields = {'$addFields':
        { 'transaction_year': { '$year': "$TransactionDate" }}}

notna = {'$match': {'SupplierInvoiceNumber': {'$ne' : np.nan}}}

lookup = {'$lookup':{
       'from': 'Purchasing_Suppliers',
       'localField': 'SupplierID',
       'foreignField': 'SupplierID',
       'as': 'Purchasing'}}

unwind = {'$unwind': '$Purchasing'}
   
    
    
project = {"$project": {
        "transaction_year":1,
        "Purchasing.SupplierName":1,
        "TransactionAmount":1,
    "TransactionID":1
    }
}

group={'$group':{
        '_id':{'transaction_year':'$transaction_year','name':'$Purchasing.SupplierName'},
        'sum':{'$sum':'$TransactionAmount'}}}

sort =  {'$sort': {'_id.name':1}}

pipeline=[convert,addfields,notna,lookup,unwind,project,group,sort]

r_3=db.Purchasing_SupplierTransactions.aggregate(pipeline)
result=list(r_3)
result

[{'_id': {'transaction_year': 2016, 'name': 'A Datum Corporation'},
  'sum': 27525.3},
 {'_id': {'transaction_year': 2013, 'name': 'Contoso, Ltd.'}, 'sum': 360.53},
 {'_id': {'transaction_year': 2013, 'name': 'Fabrikam, Inc.'},
  'sum': 60480866.71},
 {'_id': {'transaction_year': 2015, 'name': 'Fabrikam, Inc.'},
  'sum': 339016809.9},
 {'_id': {'transaction_year': 2016, 'name': 'Fabrikam, Inc.'},
  'sum': 188005969.8},
 {'_id': {'transaction_year': 2014, 'name': 'Fabrikam, Inc.'},
  'sum': 192272508.9},
 {'_id': {'transaction_year': 2013, 'name': 'Graphic Design Institute'},
  'sum': 7462.45},
 {'_id': {'transaction_year': 2013, 'name': 'Litware, Inc.'},
  'sum': 9790895.67},
 {'_id': {'transaction_year': 2015, 'name': 'Litware, Inc.'},
  'sum': 145851271.21},
 {'_id': {'transaction_year': 2014, 'name': 'Litware, Inc.'},
  'sum': 66462358.23},
 {'_id': {'transaction_year': 2016, 'name': 'Litware, Inc.'},
  'sum': 83928402.35},
 {'_id': {'transaction_year': 2013, 'name': 'Northwind Elec

## Partnerships:

#### Would like to know what's the most common payment type?

In [14]:
unwind = {'$unwind': '$Payment_Method'}

groupby = { "$group": { "_id": {'id':"$Payment_Method.PaymentMethodID", 'name': "$Payment_Method.PaymentMethodName"}, "count": { "$sum": 1 } } }

notna = {'$match': { 'count': {'$ne' : None} }}

sortdes = { "$sort": { "count": -1 }}

limit = { '$limit': 1}

pipeline = [unwind, groupby, notna, sortdes, limit]

r = db.Transactions.aggregate(pipeline)
           
list(r)

[{'_id': {'id': 4, 'name': 'EFT'}, 'count': 16447}]

#### Which supplier of Novelty Goods Supplier as the most transactions?


In [15]:
lookup = {'$lookup': { 'from': "Purchasing_Suppliers",
        'localField': "SupplierID",
        'foreignField': "SupplierID",
        'as': "sup_info"
    }}

unwind = {'$unwind': '$sup_info'}

owwwant_query = {
    '$project':{
        "_id": 0,
        "sup_info.Supplier_Category.SupplierCategoryName": 1,
        "sup_info.SupplierName":1,
        "SupplierTransactionID":1
    }
}

match = {'$match': { 'sup_info.Supplier_Category.SupplierCategoryName': {'$eq' : 'Novelty Goods Supplier'} }}

groupby = { "$group": { "_id": "$sup_info.SupplierName", "count": { "$sum": 1 } } }

sortdes = { "$sort": { "count": -1 }}

limit = { '$limit': 1}

pipeline = [lookup, unwind, owwwant_query,match, groupby, sortdes, limit]

r = db.Purchasing_SupplierTransactions.aggregate(pipeline)
           
list(r)

[{'_id': 'Graphic Design Institute', 'count': 16}]

## The marketing team:


#### Want to make an appreciation post and needs the name of the sales person with the most invoices in 2013 (person who's customers brought the most money)?

In [ ]:
groupinv = { "$group": { "_id": "$InvoiceID", "TotalInv": { "$sum": '$TotalItemPrice' } } }

project = {'$project': {'_id':1,
                       'TotalInv': 1}}

looktoinv = {'$lookup': { 'from': "Sales_Invoices",
        'localField': "_id",
        'foreignField': "InvoiceID",
        'as': "invoice_info"
    }}

unwind ={"$unwind":"$invoice_info"}

project2 = {'$project': {'_id':0,
                       'invoice_info.InvoiceID': 1,
                        'invoice_info.SalespersonPersonID':1,
                        'TotalInv':1,
                        'invoice_info.InvoiceDate':1}}

convert = { "$addFields": {
    "InvDate": { "$convert": { 
                "input": "$invoice_info.InvoiceDate", 
                "to": "date"}}}}

addf = { "$addFields": {'year' : { '$year' : '$InvDate'}}}

match = {'$match': {'year': {'$eq' : 2013}}}

group_person = {'$group':{
    "_id": "$invoice_info.SalespersonPersonID",
    "TotalInvoices": { "$addToSet":"$TotalInv" }
}}

project3 = {'$project': { '_id': 0,
                         "invoice_info.SalespersonPersonID":1,
                        "MoneyPerson": { '$sum': "$TotalInv" }}}

sortdesc = {'$sort': {'MoneyPerson':-1}}

limit ={"$limit":1}

looktopers = {'$lookup': { 'from': "Project_WWI_Application_People",
        'localField': "invoice_info.SalespersonPersonID",
        'foreignField': "PersonID",
        'as': "person_info"
    }}

unwind2 = {"$unwind":"$person_info"}

finproj = {'$project': { '_id': 0,
                         'PersonName': "$person_info.FullName",
                        "MoneyPerson":1}}


pipeline = [groupinv,project, looktoinv,unwind,project2,convert,addf, match,group_person, project3,
                                      sortdesc, limit, looktopers,unwind2,finproj]

r = db.Sales_InvoiceLines.aggregate(pipeline)
           
list(r)